In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
print ('Files in Drive:')

!ls /content/gdrive/'My Drive'

Files in Drive:
'Colab Notebooks'


In [0]:
import os
import pprint
import tensorflow as tf

In [30]:
!pip install kashgari

In [31]:
from kashgari.utils.logger import init_logger
init_logger()

from kashgari.tasks.seq_labeling import BLSTMCRFModel

----------------------
[2019-04-20 18:53:31,317] INFO  - logging init finished


# Data

In [32]:
import pandas as pd
#df1 = pd.read_csv('drive/Colab Notebooks/data/ner/Lenta_Crimes_Tagged.csv')
df1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/Lighttag_0-100_parsed_data_str.csv')
df1.head()

,tags,texts
0,O O O O O O O O O O OFF O O O O O O O O DAT DA...,Федеральное налоговое ведомство США по итогам ...
1,O O O O O O O O O O O O O O O O O O O O O O O ...,Приволжское следственное управление на транспо...
2,O O O O O O O O O OFF OFF OFF O O O O O O O O ...,"Уголовное дело , фигурантом которого был генер..."
3,O O O O O O O O O OFF O O O O O O O O O O O O ...,Московская налоговая инспекция проводит новую ...
4,O O O O O OFF OFF O O O O O LOC O O O O O O O ...,Российское следствие объявило украинского бизн...


In [0]:
list_of_tags = []
for i in range(0, len(df1['tags'])):
  list_of_tags.append(df1['tags'][i].split(' '))
  
list_of_texts = []
for i in range(0, len(df1['texts'])):
  list_of_texts.append(df1['texts'][i].split(' '))


In [0]:
for i in range(0, len(list_of_texts)):
  if len(list_of_tags[i])!=len(list_of_texts[i]):
    print('i: ',i)
    print('diff: ',len(list_of_texts[i])- len(list_of_tags[i]))

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(list_of_texts, list_of_tags, test_size=0.2, random_state=42) 

# Embeddings

In [0]:
#folder = 'multi_cased_L-12_H-768_A-12'
#download_url = 'https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip'  # ссылка на скачивание модели

#print('Downloading model...')
#zip_path = '{}.zip'.format(folder)
#!test -d $folder || (wget $download_url && unzip $zip_path)

#config_path = folder+'/bert_config.json'
#checkpoint_path = folder+'/bert_model.ckpt'
#vocab_path = folder+'/vocab.txt'

In [37]:
folder = 'rubert_cased_L-12_H-768_A-12_v1'
download_url = '/content/gdrive/My Drive/Colab Notebooks/data/rubert_cased_L-12_H-768_A-12_v1.tar.gz'

print('Unpacking model...')
zip_path = '{}.tar.gz'.format(folder)
!test -d $folder || (tar xvzf '/content/gdrive/My Drive/Colab Notebooks/data/rubert_cased_L-12_H-768_A-12_v1.tar.gz')

config_path = folder+'/bert_config.json'
checkpoint_path = folder+'/bert_model.ckpt'
vocab_path = folder+'/vocab.txt'

Unpacking model...


In [38]:
from kashgari.embeddings import BERTEmbedding
embedding = BERTEmbedding('rubert_cased_L-12_H-768_A-12_v1', 300)

[2019-04-20 18:53:37,628] INFO  - loading bert model from rubert_cased_L-12_H-768_A-12_v1



# Callbacks

In [0]:
from keras.callbacks import EarlyStopping

In [0]:
from keras.callbacks import ModelCheckpoint
# checkpoint
filepath="/content/gdrive/My Drive/Colab Notebooks/output_NER(custom) kashgari_rubert/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_loss', patience=5)
callbacks_list = [checkpoint, es]
#monitor = [f1]

# Training

In [43]:
# train model
#model = BLSTMCRFModel()
model = BLSTMCRFModel(embedding)
model.fit(X_train, y_train, batch_size = 50, epochs = 50, verbose=0, callbacks=callbacks_list)
  #, validation_steps=100, x_validate=X_test, y_validate=y_test)

#save model
model.save('/content/gdrive/My Drive/Colab Notebooks/output_NER(custom) kashgari_rubert/')

[2019-04-20 18:56:31,330] DEBUG - bert embedding no need to build token2idx with corpus
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 300)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 300)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 300, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 300, 768)     1536        Input-Segment[0][0]              
_____________________

In [44]:
#train model
model.fit(X_train, y_train, batch_size = 24, epochs = 20, verbose=0, callbacks=callbacks_list)
          #, x_validate=X_test, y_validate=y_test, validation_steps=100)

#save model
model.save('/content/gdrive/My Drive/Colab Notebooks/output_NER(custom) kashgari_rubert/')

[2019-04-20 18:58:48,441] DEBUG - bert embedding no need to build token2idx with corpus
Epoch 1/20
3/3 [==============================] - 5s 2s/step - loss: 0.0609 - crf_accuracy: 0.9787
Epoch 2/20
3/3 [==============================] - 5s 2s/step - loss: 0.0559 - crf_accuracy: 0.9801
Epoch 3/20
3/3 [==============================] - 5s 2s/step - loss: 0.0566 - crf_accuracy: 0.9807
Epoch 4/20
3/3 [==============================] - 5s 2s/step - loss: 0.0540 - crf_accuracy: 0.9805
Epoch 5/20
3/3 [==============================] - 5s 2s/step - loss: 0.0494 - crf_accuracy: 0.9828
Epoch 6/20
3/3 [==============================] - 5s 2s/step - loss: 0.0468 - crf_accuracy: 0.9821
Epoch 7/20
3/3 [==============================] - 6s 2s/step - loss: 0.0488 - crf_accuracy: 0.9821
Epoch 8/20
3/3 [==============================] - 5s 2s/step - loss: 0.0458 - crf_accuracy: 0.9850
Epoch 9/20
3/3 [==============================] - 5s 2s/step - loss: 0.0422 - crf_accuracy: 0.9856
Epoch 10/20
3/3 [====

In [45]:
#train model
model.fit(X_train, y_train, batch_size = 5, epochs = 20, verbose=0, callbacks=callbacks_list)
          #, x_validate=X_test, y_validate=y_test, validation_steps=100)

#save model
model.save('/content/gdrive/My Drive/Colab Notebooks/output_NER(custom) kashgari_rubert/')

[2019-04-20 19:00:39,534] DEBUG - bert embedding no need to build token2idx with corpus
Epoch 1/20
16/16 [==============================] - 22s 1s/step - loss: 0.0308 - crf_accuracy: 0.9886
Epoch 2/20
16/16 [==============================] - 20s 1s/step - loss: 0.0314 - crf_accuracy: 0.9883
Epoch 3/20
16/16 [==============================] - 20s 1s/step - loss: 0.0334 - crf_accuracy: 0.9875
Epoch 4/20
16/16 [==============================] - 20s 1s/step - loss: 0.0333 - crf_accuracy: 0.9867
Epoch 5/20
16/16 [==============================] - 21s 1s/step - loss: 0.0277 - crf_accuracy: 0.9892
Epoch 6/20
16/16 [==============================] - 20s 1s/step - loss: 0.0189 - crf_accuracy: 0.9940
Epoch 7/20
16/16 [==============================] - 20s 1s/step - loss: 0.0195 - crf_accuracy: 0.9933
Epoch 8/20
16/16 [==============================] - 21s 1s/step - loss: 0.0160 - crf_accuracy: 0.9948
Epoch 9/20
16/16 [==============================] - 20s 1s/step - loss: 0.0132 - crf_accuracy: 0

# Evaluation

In [46]:
model.evaluate(X_test, y_test)

             precision    recall  f1-score   support

        DAT     0.5714    0.2667    0.3636        15
        LOC     0.3333    0.0714    0.1176        14
        SUF     0.0000    0.0000    0.0000         1

avg / total     0.4413    0.1667    0.2367        30



'             precision    recall  f1-score   support\n\n        DAT     0.5714    0.2667    0.3636        15\n        LOC     0.3333    0.0714    0.1176        14\n        SUF     0.0000    0.0000    0.0000         1\n\navg / total     0.4413    0.1667    0.2367        30\n'

# Predictions

In [52]:
ans = []
for i in range(0,10):
    print(model.predict(X_test[i]))
    ans.append(model.predict(X_test[i]))
    print(X_test[i])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [53]:
for i in range(0, len(ans)):
  for j in range(0, len(ans[i])):
    if y_test[i][j] != 'O':
      print('Word ',X_test[i][j])
      print('Answer ',ans[i][j])
      print('Original tag ',y_test[i][j])
      print('\n')

  print('\n')



Word  Ярославля
Answer  O
Original tag  LOC


Word  Евгений
Answer  OFF
Original tag  OFF


Word  Урлашов
Answer  OFF
Original tag  OFF


Word  Дмитрий
Answer  O
Original tag  OFF


Word  Донсков
Answer  O
Original tag  OFF


Word  в
Answer  O
Original tag  DAT


Word  2012–2013
Answer  O
Original tag  DAT


Word  годах
Answer  O
Original tag  DAT


Word  Радострой
Answer  OFF
Original tag  SUF


Word  Ярославля
Answer  O
Original tag  LOC




Word  Мишеля
Answer  OFF
Original tag  OFF


Word  Платини
Answer  OFF
Original tag  OFF


Word  Йозефа
Answer  OFF
Original tag  OFF


Word  Блаттера
Answer  OFF
Original tag  OFF


Word  Блаттера
Answer  O
Original tag  OFF


Word  Блаттер
Answer  O
Original tag  OFF




Word  Лабытнанги
Answer  LOC
Original tag  LOC


Word  Василий
Answer  OFF
Original tag  OFF


Word  Лукаш
Answer  OFF
Original tag  OFF


Word  Василий
Answer  O
Original tag  OFF


Word  Лукаш
Answer  OFF
Original tag  OFF


Word  Лукаша
Answer  OFF
Original tag  OFF








In [54]:
for i in range(0, len(ans)):
  for j in range(0, len(ans[i])):
    if ans[i][j] != 'O':
      print('Word ',X_test[i][j])
      print('Answer ',ans[i][j])
      print('Original tag ',y_test[i][j])
      print('\n')

  print('\n')



Word  Евгений
Answer  OFF
Original tag  OFF


Word  Урлашов
Answer  OFF
Original tag  OFF


Word  Радострой
Answer  OFF
Original tag  SUF


Word  Урлашов
Answer  OFF
Original tag  O




Word  Мишеля
Answer  OFF
Original tag  OFF


Word  Платини
Answer  OFF
Original tag  OFF


Word  Йозефа
Answer  OFF
Original tag  OFF


Word  Блаттера
Answer  OFF
Original tag  OFF


Word  Платини
Answer  OFF
Original tag  O


Word  Платини
Answer  OFF
Original tag  O




Word  Лабытнанги
Answer  LOC
Original tag  LOC


Word  Василий
Answer  OFF
Original tag  OFF


Word  Лукаш
Answer  OFF
Original tag  OFF


Word  Лукаш
Answer  OFF
Original tag  OFF


Word  Лукаша
Answer  OFF
Original tag  OFF






Word  Екатерине
Answer  OFF
Original tag  OFF


Word  Вороновской
Answer  OFF
Original tag  OFF


Word  Денис
Answer  OFF
Original tag  OFF


Word  Сугробов
Answer  OFF
Original tag  OFF


Word  Вороновская
Answer  OFF
Original tag  O


Word  в
Answer  DAT
Original tag  DAT


Word  октябре
Answer  DAT
Orig

In [55]:
for i in range(0, len(ans)):
  for j in range(0, len(ans[i])):
    if y_test[i][j] != 'O' and ans[i][j] == 'O':
      print('Word ',X_test[i][j])
      print('Answer ',ans[i][j])
      print('Original tag ',y_test[i][j])
      print('\n')

  print('\n')



Word  Ярославля
Answer  O
Original tag  LOC


Word  Дмитрий
Answer  O
Original tag  OFF


Word  Донсков
Answer  O
Original tag  OFF


Word  в
Answer  O
Original tag  DAT


Word  2012–2013
Answer  O
Original tag  DAT


Word  годах
Answer  O
Original tag  DAT


Word  Ярославля
Answer  O
Original tag  LOC




Word  Блаттера
Answer  O
Original tag  OFF


Word  Блаттер
Answer  O
Original tag  OFF




Word  Василий
Answer  O
Original tag  OFF






Word  Химки
Answer  O
Original tag  LOC


Word  Мираб
Answer  O
Original tag  OFF


Word  Рябушкин
Answer  O
Original tag  OFF




Word  Санкт-Петербурга
Answer  O
Original tag  LOC




Word  Сергиева
Answer  O
Original tag  LOC


Word  Посада
Answer  O
Original tag  LOC


Word  Викторе
Answer  O
Original tag  OFF


Word  Букине
Answer  O
Original tag  OFF


Word  Сергиева
Answer  O
Original tag  LOC


Word  Посада
Answer  O
Original tag  LOC




Word  поселке
Answer  O
Original tag  LOC


Word  Московский
Answer  O
Original tag  LOC


Word  Пле

In [56]:
for i in range(0, len(ans)):
  for j in range(0, len(ans[i])):
    if ans[i][j] != 'O' and y_test[i][j] == 'O':
      print('Word ',X_test[i][j])
      print('Answer ',ans[i][j])
      print('Original tag ',y_test[i][j])
      print('\n')

  print('\n')



Word  Урлашов
Answer  OFF
Original tag  O




Word  Платини
Answer  OFF
Original tag  O


Word  Платини
Answer  OFF
Original tag  O








Word  Вороновская
Answer  OFF
Original tag  O


Word  Рябушкин
Answer  OFF
Original tag  O




Word  с
Answer  DAT
Original tag  O


Word  января
Answer  DAT
Original tag  O


Word  2011
Answer  DAT
Original tag  O


Word  по
Answer  DAT
Original tag  O


Word  март
Answer  DAT
Original tag  O


Word  2014
Answer  DAT
Original tag  O


Word  года
Answer  DAT
Original tag  O










